In [ ]:
!apt-get install build-essential python3-dev python3-numpy libhdf4-dev -y
!pip install pyhdf
!pip install openturns
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.lines as mlines
import tables
import h5py
import numpy as np
import sys
from pyhdf.SD import SD, SDC
import pandas as pd
import time
import calendar
import matplotlib.pyplot as plt
import datetime
import os
import openturns as ot
import datetime
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/BadgerX Data"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python3-numpy is already the newest version (1:1.13.3-2ubuntu1).
python3-numpy set to manually installed.
python3-dev is already the newest version (3.6.7-1~18.04).
The following additional packages will be installed:
  libhdf4-0
Suggested packages:
  libhdf4-doc hdf4-tools
The following NEW packages will be installed:
  libhdf4-0 libhdf4-dev
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 716 kB of archives.
After this operation, 3,154 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhdf4-0 amd64 4.2.13-2 [301 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libhdf4-dev amd64 4.2.13-2 [415 kB]
Fetched 716 kB in 2s (317 kB/s)
Selecting previously unselected package libhdf4-0.
(Reading database ... 144793 files and directories curren

In [ ]:
import csv
import numpy as np

ave = np.array(list(csv.reader(open("narr2010ave.csv"))), dtype=float)
for i in range(1, 10):
  temp = np.array(list(csv.reader(open("narr201"+str(i) + "ave.csv"))),dtype=float)
  print(temp.shape)
  ave = np.concatenate((ave, temp), axis=0)


print(ave.shape)

(365, 57600)
(366, 57600)
(365, 57600)
(365, 57600)
(365, 57600)
(366, 57600)
(365, 57600)
(365, 57600)
(365, 57600)
(3652, 57600)


In [ ]:
fin = []
ave = pd.DataFrame(ave)

for day in range(3652):hum
  krig_data = ave.iloc[day]
  krig_data_train = np.where(hum != 0) #np.where(hum != 0)
  krig_data_test = np.where(hum == 0)
  krig_data_train = krig_data[krig_data_train[0]] #gets values
  krig_data_test = krig_data[krig_data_test[0]]
  lons = [krig_data_train.index[i] % 120 for i in range(len(krig_data_train))]
  lats = [(krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120 for i in range(len(krig_data_train))]
  c = [np.array(i) for i in zip(lats, lons)]
  d = [[krig_data_train.iloc[i]] for i in range(len(krig_data_train))]
  coordinates_train = ot.Sample(c)
  aod_train = ot.Sample(d)
  inputDimension = 2
  basis = ot.ConstantBasisFactory(inputDimension).build()
  covarianceModel = ot.SquaredExponential([1.]*inputDimension, [1.0])
  algo = ot.KrigingAlgorithm(coordinates_train, aod_train, covarianceModel, basis)
  algo.run()
  result = algo.getResult()
  krigingMetamodel = result.getMetaModel()
  lons_test = [krig_data_test.index[i] % 120 for i in range(len(krig_data_test))]
  lats_test = [(krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120 for i in range(len(krig_data_test))]
  c = [np.array(i) for i in zip(lats_test, lons_test)]
  kriged_data_pred = krigingMetamodel(c)
  kriged_data_pred = pd.Series(np.array(kriged_data_pred).reshape(-1))
  kriged_map_arr = np.zeros((120, 120))

  for i in range(len(krig_data_test.index)):
    kriged_map_arr[(int)((krig_data_test.index[i] - (krig_data_test.index[i] % 120)) / 120), (int)(krig_data_test.index[i] % 120)] = kriged_data_pred[i]
    
  for i in range(len(krig_data_train.index)):
    kriged_map_arr[(int)((krig_data_train.index[i] - (krig_data_train.index[i] % 120)) / 120), (int)(krig_data_train.index[i] % 120)] = krig_data_train[krig_data_train.index[i]]

  fin.append(kriged_map_arr)
  print(day)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


KeyboardInterrupt: ignored